In [1]:
from swiper2.schedule_experiments import RegularTSchedule
from swiper2.device_manager import DeviceManager
from swiper2.window_builder import SlidingWindowBuilder

In [2]:
d=15
test_experiment = RegularTSchedule(10, 5)
manager = DeviceManager(d, test_experiment.schedule)

In [3]:
sliding_builder = SlidingWindowBuilder(d)

In [4]:
for i in range(10):
    new_round = manager.get_next_round([])
    new_windows = sliding_builder.build_windows(new_round)

TypeError: unsupported operand type(s) for -: 'SyndromeRound' and 'SyndromeRound'